In [1]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.feature import *
from pyspark.ml import Pipeline
from pyspark.ml.classification import *
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql import SparkSession
from pyspark.ml.classification import LinearSVC
from pyspark.ml.functions import vector_to_array
import pandas as pd
import numpy as np
import os
import pyarrow.parquet as pq
from pyspark.ml.feature import StandardScaler
from pyspark.sql.functions import mean as _mean, stddev as _stddev, col
from math import *
from pyspark.sql.window import Window


In [2]:
spark = SparkSession.builder.master("local[1]")\
          .appName("random")\
          .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/22 17:46:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/22 17:46:29 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/05/22 17:46:29 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/05/22 17:46:29 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/05/22 17:46:29 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/05/22 17:46:29 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.


In [39]:
df = spark.read  \
     .option("delimiter", ";") \
     .option("header", "true") \
     .csv("/Users/youssouf/Downloads/ann_lagg_20220323.csv") \
     .dropDuplicates()

In [40]:
df.show(5)

+----------+----------------+------------+----------------+------------------+----------------+-------------------------+-----------------+------------+--------------------------+-------------------+--------------+------------------------+--------------------+------------+----------------+
|COMPTE_PDV|COMPTE_REVENDEUR|TETE_DE_PONT|DATE_TRANSACTION|   USER_GRADE_NAME|USER_DOMAIN_CODE|VOLUME_TOTAL_TRANSACTIONS|MONTANT_UVE_TOTAL|DAY_INITIATE|MONTANT_TRANSACTION_ANNULE|MONTANT_UVE_CORRIGE|COMMISSION_TMP|COMMISSION_GROSSISTE_TTC|COMMISSION_AGENT_TTC|STATUT_RIPSI|COMMISSION_AGENT|
+----------+----------------+------------+----------------+------------------+----------------+-------------------------+-----------------+------------+--------------------------+-------------------+--------------+------------------------+--------------------+------------+----------------+
|0779070018|      0779070069|  0758023975|      2022-03-22|PDVTYPE2 ERECHARGE|        GRSTYPE2|                       32|      

In [41]:
df.count()

32702

In [42]:
df.printSchema()

root
 |-- COMPTE_PDV: string (nullable = true)
 |-- COMPTE_REVENDEUR: string (nullable = true)
 |-- TETE_DE_PONT: string (nullable = true)
 |-- DATE_TRANSACTION: string (nullable = true)
 |-- USER_GRADE_NAME: string (nullable = true)
 |-- USER_DOMAIN_CODE: string (nullable = true)
 |-- VOLUME_TOTAL_TRANSACTIONS: string (nullable = true)
 |-- MONTANT_UVE_TOTAL: string (nullable = true)
 |-- DAY_INITIATE: string (nullable = true)
 |-- MONTANT_TRANSACTION_ANNULE: string (nullable = true)
 |-- MONTANT_UVE_CORRIGE: string (nullable = true)
 |-- COMMISSION_TMP: string (nullable = true)
 |-- COMMISSION_GROSSISTE_TTC: string (nullable = true)
 |-- COMMISSION_AGENT_TTC: string (nullable = true)
 |-- STATUT_RIPSI: string (nullable = true)
 |-- COMMISSION_AGENT: string (nullable = true)



In [ ]:

df.coalesce(1).write.mode("overwrite").format("csv").option("header", "true").option("delimiter", ";").save("/Users/youssouf/Downloads/annulation_20220322_2_ff.csv")



In [43]:
df = df.na.fill(value = "2022-03-22", subset = ["DAY_INITIATE"])

# na.fill(value = "0", subset = ["MONTANT_UVE_CORRIGE"])

In [9]:
df.write.mode("overwrite").format("parquet").save("/Users/youssouf/Downloads/annulation_20220322_3.parquet")


In [44]:
# .drop("MONTANT_TRANSACTION_ANNULE", "USER_GRADE_NAME", "USER_DOMAIN_CODE", "COMMISSION_TMP") \

df = df.drop("COMMISSION_TMP") \
.withColumnRenamed("COMMISSION_GROSSISTE_TTC", "COMMISSION_GROSSISTE_FINAL") \
.withColumn("COMMISSION_GROSSISTE_REEL", col("COMMISSION_GROSSISTE_FINAL")) \
.withColumnRenamed("COMMISSION_AGENT", "COMMISSION_AGENT_FINAL") \
.withColumn("COMMISSION_AGENT_REEL", col("COMMISSION_AGENT_FINAL")) \



In [45]:
df.printSchema()

root
 |-- COMPTE_PDV: string (nullable = true)
 |-- COMPTE_REVENDEUR: string (nullable = true)
 |-- TETE_DE_PONT: string (nullable = true)
 |-- DATE_TRANSACTION: string (nullable = true)
 |-- USER_GRADE_NAME: string (nullable = true)
 |-- USER_DOMAIN_CODE: string (nullable = true)
 |-- VOLUME_TOTAL_TRANSACTIONS: string (nullable = true)
 |-- MONTANT_UVE_TOTAL: string (nullable = true)
 |-- DAY_INITIATE: string (nullable = false)
 |-- MONTANT_TRANSACTION_ANNULE: string (nullable = true)
 |-- MONTANT_UVE_CORRIGE: string (nullable = true)
 |-- COMMISSION_GROSSISTE_FINAL: string (nullable = true)
 |-- COMMISSION_AGENT_TTC: string (nullable = true)
 |-- STATUT_RIPSI: string (nullable = true)
 |-- COMMISSION_AGENT_FINAL: string (nullable = true)
 |-- COMMISSION_GROSSISTE_REEL: string (nullable = true)
 |-- COMMISSION_AGENT_REEL: string (nullable = true)



In [2]:
from elasticsearch import Elasticsearch
import json


In [4]:
# create a client instance of the library
elastic_client = Elasticsearch(["10.242.71.153:9200"])

# make an API call to the Elasticsearch cluster to get documents
result = elastic_client.search(index='random_commission_fixe_20220403', body={}, size=99)

<ipython-input-4-2f1c6d623cef>:5: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  result = elastic_client.search(index='random_commission_fixe_20220403', body={}, size=99)


UnsupportedProductError: The client noticed that the server is not Elasticsearch and we do not support this unknown product

In [46]:

df.coalesce(1).write.mode("overwrite").format("csv").option("header", "true").option("delimiter", ";").save("/Users/youssouf/Downloads/annulation_20220322_2_ff.csv")



In [10]:
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import os

In [ ]:
def From_pyparquet_to_pandascsv() :
    df = pq.read_table("/Users/youssouf/Downloads/annulation_20220322_1.parquet").to_pandas()
    csv_save_path = "/Users/youssouf/Documents/PythonCOde/annulation_20220322_1.csv"
    df.to_csv(csv_save_path, index = False, sep = ";" )
    return os.path.isfile(csv_save_path)

In [12]:
from datetime import datetime
from elasticsearch import Elasticsearch
es = Elasticsearch("https://10.242.71.153:9200",use_ssl=False)

doc = {
    'author': 'diakite',
    'text': 'Elasticsearch: cool. bonsai cool.',
    'timestamp': datetime.now(),
}

resp = es.index(index="random_commission_fixe_20220403", id=1, document=doc)
print(resp['result'])

resp = es.get(index="random_commission_fixe_20220403", id=1)
print(resp['_source'])

es.indices.refresh(index="random_commission_fixe_20220403")

resp = es.search(index="random_commission_fixe_20220403", query={"match_all": {}})
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print("%(timestamp)s %(author)s: %(text)s" % hit["_source"])

SSLError: ConnectionError([SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1123)) caused by: SSLError([SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1123))

In [4]:
df = spark.read  \
     .option("delimiter", ";") \
     .option("header", "true") \
     .csv("/Users/diakite/Downloads/Telegram/FICHIER_ANNULATION_TEST_20220423.csv") \
     .dropDuplicates()

In [5]:
df.show(5)

+----------------+----------+----------------+------------+-------------------------+-----------------+------------+-------------------+----------------------+---------------------+--------------------------+-------------------------+------------+--------------+------------------+------------------------+--------------------------------+--------------------+----------------+---------------+--------------------+--------------------+
|DATE_TRANSACTION|COMPTE_PDV|COMPTE_REVENDEUR|TETE_DE_PONT|VOLUME_TOTAL_TRANSACTIONS|MONTANT_UVE_TOTAL|DAY_INITIATE|MONTANT_UVE_CORRIGE|COMMISSION_AGENT_FINAL|COMMISSION_AGENT_REEL|COMMISSION_GROSSISTE_FINAL|COMMISSION_GROSSISTE_REEL|STATUT_RIPSI|RELIQUAT_AGENT|RELIQUAT_GROSSISTE|COMMISSION_AGENT_CORRIGE|COMMISSION_GROSSISTE_TTC_CORRIGE|     USER_GRADE_NAME|USER_DOMAIN_CODE|USER_GRADE_CODE|        ID_REVENDEUR|     ID_TETE_DE_PONT|
+----------------+----------+----------------+------------+-------------------------+-----------------+------------+------------

In [7]:
df.filter(col("MONTANT_UVE_TOTAL") == 0).show()

+----------------+----------+----------------+------------+-------------------------+-----------------+------------+-------------------+----------------------+---------------------+--------------------------+-------------------------+------------+--------------+------------------+------------------------+--------------------------------+---------------+----------------+---------------+------------+---------------+
|DATE_TRANSACTION|COMPTE_PDV|COMPTE_REVENDEUR|TETE_DE_PONT|VOLUME_TOTAL_TRANSACTIONS|MONTANT_UVE_TOTAL|DAY_INITIATE|MONTANT_UVE_CORRIGE|COMMISSION_AGENT_FINAL|COMMISSION_AGENT_REEL|COMMISSION_GROSSISTE_FINAL|COMMISSION_GROSSISTE_REEL|STATUT_RIPSI|RELIQUAT_AGENT|RELIQUAT_GROSSISTE|COMMISSION_AGENT_CORRIGE|COMMISSION_GROSSISTE_TTC_CORRIGE|USER_GRADE_NAME|USER_DOMAIN_CODE|USER_GRADE_CODE|ID_REVENDEUR|ID_TETE_DE_PONT|
+----------------+----------+----------------+------------+-------------------------+-----------------+------------+-------------------+----------------------+-----

In [14]:
df.last()

AttributeError: 'DataFrame' object has no attribute 'last'